In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
from spyne.application import Application
from spyne.decorator import srpc
from spyne.service import ServiceBase
from spyne.model.primitive import Integer
from spyne.model.primitive import Unicode
from spyne.model.complex import Iterable
from spyne.protocol.http import HttpRpc
from spyne.protocol.json import JsonDocument
from spyne.server.wsgi import WsgiApplication
import threading
import hashlib
import redis

class DiminishedTriadService(ServiceBase):

    def __init__(self):
        print "I am being instantiated !!!!"
        
    @srpc(Unicode, _returns=Unicode)
    def get(key):
        backs = getNextNBackends(key)
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        print "lenght of backs : ", len(backs)
        print "backs : ", backs
        print "global map in get():", hash_to_back_map
        for back in backs:
            addr = back.split(":")[0]
            port = back.split(":")[1]
            try:
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.get(key)
            except:
                print "failed to get key: ", key ," from addr: ",addr," port: ", port
                continue
            
    @srpc(Unicode,Unicode, _returns=Unicode)
    def set(key,value):
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        backs = getNextNBackends(key)
        print "setting key :", key, " to value :", value
        print "lenght of backs : ", len(backs)
        print "backs : ", backs
        print "global map in set():", hash_to_back_map
        for back in backs:
            addr = back.split(":")[0]
            port = back.split(":")[1]
            try:
                r = redis.StrictRedis(host=addr, port=port, db=0)
                r.set(key,value)
            except:
                print "failed to set key: ", key ," to addr: ",addr," port: ", port
                continue
        return OK
    
    @srpc(Unicode, _returns=Unicode)
    def update_hash_to_back_map(addrs):
        try:
            lock.acquire()
            new_hash_to_back_map = {}
            for addr in addrs.split(","):
                new_hash_to_back_map[hash(addr)] = addr
            hash_to_back_map.clear()
            for k in new_hash_to_back_map:
                hash_to_back_map[k] = new_hash_to_back_map[k]
        except:
            e = sys.exc_info()[0]
            print e
            return e
        finally:
            lock.release()
            return OK
    
    
    @srpc(_returns=Unicode)
    def getAOFLog():
        f = open(AOFPATH,'r')
        string = f.read()
        print "getAOF returning ", string
        return string
    
    @srpc(Unicode,_returns=Integer)
    def getStrlen(key):
        backs = getNextNBackends(key)
        for back in backs:
            try:
                addr = back.split(":")[0]
                port = back.split(":")[1]
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.get(key)
            except:
                continue
                
    @srpc(Unicode,Unicode, _returns=Integer)
    def append(key,append_str):
        backs = getNextNBackends(key)
        for back in backs:
            try:
                addr = back.split(":")[0]
                port = back.split(":")[1]
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.append(key,append_str)
            except:
                print "failed to append to key: ", key ," append_str : ", append_str ," to addr: ", addr , "port: " , port
                continue
        
    @srpc(Unicode,Unicode, _returns=Integer)
    def lPush(key,append_str):
        backs = getNextNBackends(key)
        for back in backs:
            try:
                addr = back.split(":")[0]
                port = back.split(":")[1]
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.lpush(key,append_str)
            except:
                print "failed to lPush to key: ", key ," append_str : ", append_str ," to addr: ", addr , "port: " , port
                continue
                
    @srpc(Unicode, _returns=Integer)
    def lPop(key):
        backs = getNextNBackends(key)
        for back in backs:
            try:
                addr = back.split(":")[0]
                port = back.split(":")[1]
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.lpop(key)
            except:
                print "failed to lPop to key: ", key ," to addr: ", addr , "port: " , port
                continue
          
        
    
    
############################### End of Class DiminishedTriadService ########################################

NUMBACKENDS = 100 # this determines what we are modding by
NUMBACKUPS = 2
OK = "ok"

def hash(key):
    return int(hashlib.sha1(key).hexdigest(), 16)

lock = threading.RLock()
AOFPATH = "create-multiredis/appendonly-30001.aof"
hash_to_back_map = {
                            hash("127.0.0.1:30001"):"127.0.0.1:30001",
                            hash("127.0.0.1:30002"):"127.0.0.1:30002",
                            hash("127.0.0.1:30003"):"127.0.0.1:30003",
                            hash("127.0.0.1:30004"):"127.0.0.1:30004",
                            hash("127.0.0.1:30005"):"127.0.0.1:30005",
                            hash("127.0.0.1:30006"):"127.0.0.1:30006"
                    }


def getNextNBackends(key):
    try:
        lock.acquire()
        keys = hash_to_back_map.keys()
        print "keys : ", keys
        keys.append(hash(key))
        keys = sorted(keys)
        index = keys.index(hash(key))
        out = []
        for i in range(1,NUMBACKUPS+2):
            # the master will be the (index + 1)th element 
            # we are selecting N+1 elements starting with the master -- master + N backups
            if keys[(index + i)%len(keys)] in hash_to_back_map and hash_to_back_map[keys[(index + i)%len(keys)]] not in out:
                out.append(hash_to_back_map[keys[(index + i)%len(keys)]])
        print "getNextNBackends for key : ",key," returning : {",out,"}"
        return out
    finally:
        lock.release()

    

if __name__ == '__main__':
    from wsgiref.simple_server import make_server
#     logging.basicConfig(level=logging.DEBUG)
#     logging.getLogger('spyne.protocol.xml').setLevel(logging.DEBUG)
    application = Application([DiminishedTriadService],
        tns='DiminishedTriadService',
        in_protocol=HttpRpc(validator='soft'),
        out_protocol=JsonDocument()
    )
    wsgi_app = WsgiApplication(application)
    server = make_server('127.0.0.1', 50001, wsgi_app)
    server.serve_forever()
    
    
    

keys :  [1431641072301691499099139682934353307135755647886L, 290362888595141267489940489438156579245598215617L, 651384643548270203367746822877458697869892333467L, 509975857984240999015181754826088054568987413620L, 428609273251099003521298261629472072668501370622L, 1301405233003453791907067794044513115772318761991L]
getNextNBackends for key :  foo  returning : { ['127.0.0.1:30003', '127.0.0.1:30002', '127.0.0.1:30006'] }
setting key : foo  to value : blahfoo
lenght of backs :  3
backs :  ['127.0.0.1:30003', '127.0.0.1:30002', '127.0.0.1:30006']
global map in set(): {1431641072301691499099139682934353307135755647886L: '127.0.0.1:30004', 290362888595141267489940489438156579245598215617L: '127.0.0.1:30003', 651384643548270203367746822877458697869892333467L: '127.0.0.1:30005', 509975857984240999015181754826088054568987413620L: '127.0.0.1:30006', 428609273251099003521298261629472072668501370622L: '127.0.0.1:30002', 1301405233003453791907067794044513115772318761991L: '127.0.0.1:30001'}
keys :

127.0.0.1 - - [25/May/2015 19:53:41] "GET /set?key=foo&value=blahfoo HTTP/1.1" 200 4
127.0.0.1 - - [25/May/2015 19:53:41] "GET /set?key=boo&value=blahboo HTTP/1.1" 200 4
127.0.0.1 - - [25/May/2015 19:53:41] "GET /set?key=goo&value=blahgoo HTTP/1.1" 200 4


 [1431641072301691499099139682934353307135755647886L, 290362888595141267489940489438156579245598215617L, 651384643548270203367746822877458697869892333467L, 509975857984240999015181754826088054568987413620L, 428609273251099003521298261629472072668501370622L, 1301405233003453791907067794044513115772318761991L]
getNextNBackends for key :  goo  returning : { ['127.0.0.1:30002', '127.0.0.1:30006', '127.0.0.1:30005'] }
setting key : goo  to value : blahgoo
lenght of backs :  3
backs :  ['127.0.0.1:30002', '127.0.0.1:30006', '127.0.0.1:30005']
global map in set(): {1431641072301691499099139682934353307135755647886L: '127.0.0.1:30004', 290362888595141267489940489438156579245598215617L: '127.0.0.1:30003', 651384643548270203367746822877458697869892333467L: '127.0.0.1:30005', 509975857984240999015181754826088054568987413620L: '127.0.0.1:30006', 428609273251099003521298261629472072668501370622L: '127.0.0.1:30002', 1301405233003453791907067794044513115772318761991L: '127.0.0.1:30001'}
keys :  [1431

127.0.0.1 - - [25/May/2015 19:53:41] "GET /set?key=zoo&value=blahzoo HTTP/1.1" 200 4
127.0.0.1 - - [25/May/2015 19:58:52] "GET /set?key=a&value=a HTTP/1.1" 200 4


 [1431641072301691499099139682934353307135755647886L, 290362888595141267489940489438156579245598215617L, 651384643548270203367746822877458697869892333467L, 509975857984240999015181754826088054568987413620L, 428609273251099003521298261629472072668501370622L, 1301405233003453791907067794044513115772318761991L]
getNextNBackends for key :  a  returning : { ['127.0.0.1:30001', '127.0.0.1:30004', '127.0.0.1:30003'] }
setting key : a  to value : a
lenght of backs :  3
backs :  ['127.0.0.1:30001', '127.0.0.1:30004', '127.0.0.1:30003']
global map in set(): {1431641072301691499099139682934353307135755647886L: '127.0.0.1:30004', 290362888595141267489940489438156579245598215617L: '127.0.0.1:30003', 651384643548270203367746822877458697869892333467L: '127.0.0.1:30005', 509975857984240999015181754826088054568987413620L: '127.0.0.1:30006', 428609273251099003521298261629472072668501370622L: '127.0.0.1:30002', 1301405233003453791907067794044513115772318761991L: '127.0.0.1:30001'}
keys :  [14316410723016

...

 [1431641072301691499099139682934353307135755647886L, 290362888595141267489940489438156579245598215617L, 651384643548270203367746822877458697869892333467L, 509975857984240999015181754826088054568987413620L, 428609273251099003521298261629472072668501370622L, 1301405233003453791907067794044513115772318761991L]
getNextNBackends for key :  f  returning : { ['127.0.0.1:30002', '127.0.0.1:30006', '127.0.0.1:30005'] }
lenght of backs :  3
backs :  ['127.0.0.1:30002', '127.0.0.1:30006', '127.0.0.1:30005']
global map in get(): {1431641072301691499099139682934353307135755647886L: '127.0.0.1:30004', 290362888595141267489940489438156579245598215617L: '127.0.0.1:30003', 651384643548270203367746822877458697869892333467L: '127.0.0.1:30005', 509975857984240999015181754826088054568987413620L: '127.0.0.1:30006', 428609273251099003521298261629472072668501370622L: '127.0.0.1:30002', 1301405233003453791907067794044513115772318761991L: '127.0.0.1:30001'}
keys :  [14316410723016914990991396829343533071357556

127.0.0.1 - - [25/May/2015 19:58:52] "GET /get?key=j HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:52] "GET /get?key=k HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:52] "GET /get?key=l HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:52] "GET /get?key=m HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:52] "GET /get?key=n HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:52] "GET /get?key=o HTTP/1.1" 200 3


 [1431641072301691499099139682934353307135755647886L, 290362888595141267489940489438156579245598215617L, 651384643548270203367746822877458697869892333467L, 509975857984240999015181754826088054568987413620L, 428609273251099003521298261629472072668501370622L, 1301405233003453791907067794044513115772318761991L]
getNextNBackends for key :  k  returning : { ['127.0.0.1:30003', '127.0.0.1:30002', '127.0.0.1:30006'] }
lenght of backs :  3
backs :  ['127.0.0.1:30003', '127.0.0.1:30002', '127.0.0.1:30006']
global map in get(): {1431641072301691499099139682934353307135755647886L: '127.0.0.1:30004', 290362888595141267489940489438156579245598215617L: '127.0.0.1:30003', 651384643548270203367746822877458697869892333467L: '127.0.0.1:30005', 509975857984240999015181754826088054568987413620L: '127.0.0.1:30006', 428609273251099003521298261629472072668501370622L: '127.0.0.1:30002', 1301405233003453791907067794044513115772318761991L: '127.0.0.1:30001'}
keys :  [14316410723016914990991396829343533071357556

127.0.0.1 - - [25/May/2015 19:58:52] "GET /get?key=p HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:53] "GET /get?key=q HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:53] "GET /get?key=r HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:53] "GET /get?key=s HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:53] "GET /get?key=t HTTP/1.1" 200 3
127.0.0.1 - - [25/May/2015 19:58:53] "GET /get?key=u HTTP/1.1" 200 3


 [1431641072301691499099139682934353307135755647886L, 290362888595141267489940489438156579245598215617L, 651384643548270203367746822877458697869892333467L, 509975857984240999015181754826088054568987413620L, 428609273251099003521298261629472072668501370622L, 1301405233003453791907067794044513115772318761991L]
getNextNBackends for key :  u  returning : { ['127.0.0.1:30006', '127.0.0.1:30005', '127.0.0.1:30001'] }
lenght of backs :  3
backs :  ['127.0.0.1:30006', '127.0.0.1:30005', '127.0.0.1:30001']
global map in get(): {1431641072301691499099139682934353307135755647886L: '127.0.0.1:30004', 290362888595141267489940489438156579245598215617L: '127.0.0.1:30003', 651384643548270203367746822877458697869892333467L: '127.0.0.1:30005', 509975857984240999015181754826088054568987413620L: '127.0.0.1:30006', 428609273251099003521298261629472072668501370622L: '127.0.0.1:30002', 1301405233003453791907067794044513115772318761991L: '127.0.0.1:30001'}
keys :  [14316410723016914990991396829343533071357556